In [1]:
from pycaret.classification import *
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import f1_score
import numpy as np
import os

In [ ]:
X_test = pd.read_parquet('../data/raw/X_test.parquet')
X_test = X_test.rename(columns={'ЭКСГАУСТЕР 4. ТОК РОТОРА2':'ЭКСГАУСТЕР 4. ТОК РОТОРА 2'})

X_test.info(max_cols=2)

In [2]:
y_test = pd.read_parquet('../data/raw/sample_submission_2.parquet')

In [ ]:
def get_exhauster(target):
    name = target.split('_')[1]
    number = int(name[-1])
    return number

def generate_features(X, features, window_diff=24):
    #
    X = X.copy()
    
    # Медианная температура и отклонения от нее
    temp_cols = [s for s in features if ('ТЕМПЕРАТУРА' in s) and (not s.startswith('Y'))]
    X['{0}. ВИБРАЦИЯ МЕДИАНА'.format(temp_cols[0].split('.')[0])] = X[temp_cols].median()
    
    temp_cols_diff = [f'{s} ОТКЛОНЕНИЕ' for s in features if 'ТЕМПЕРАТУРА' in s]
    X[temp_cols_diff] = X[temp_cols] - X[temp_cols].median()
    
    
    # Медианная вибрация и отклонения от нее
    vibr_cols = [s for s in features if 'ВИБРАЦИЯ' in s]
    X['{0}. ВИБРАЦИЯ МЕДИАНА'.format(vibr_cols[0].split('.')[0])] = X[vibr_cols].median()

    vibr_cols_diff = [f'{s} ОТКЛОНЕНИЕ' for s in features if 'ВИБРАЦИЯ' in s]
    X[vibr_cols_diff] = X[vibr_cols] - X[vibr_cols].median()
    
    return X

In [ ]:
# Подготовка данных
X_test_r = X_test.rolling('60T').mean()


In [ ]:
X_test[y_test.columns] = 0

In [ ]:
for target in sorted(y_test.columns):
    tn = target.replace('/', '___')
    
    if os.path.exists(f'../models/task2/{tn}.pkl'):
        print(target)
        
        exh_n = get_exhauster(target)
        features = [s for s in  X_test_r.columns if f'ЭКСГАУСТЕР {exh_n}' in s]
        X_test_r_f = generate_features(X_test_r, features)
        
        model = load_model(f'../models/task2/{tn}', verbose=False)
        pr = predict_model(model, X_test_r_f)
        X_test[target] = pr['prediction_label']
        
        plt.figure()
        pr['prediction_label'].resample('1D').max().plot()
        plt.title(target)
        plt.savefig(f'../figs/task2/{tn}.png')
        plt.close()


In [ ]:
X_test[y_test.columns].drop(columns='DT').reset_index().to_parquet('../data/submissions/submission_2.parquet')